In [1]:
from IPython.display import Markdown, display
from mento import cm, kN, m

In [2]:
b=20*cm
h=50*cm
M_u=50*kN*m # type: ignore # noqa: F821
ØM_n=55*kN*m # type: ignore # noqa: F821
FU_b=M_u/ØM_n
rebar_b="3Ø12"
A_s=1.13*cm**2*3
display(Markdown(f"Viga 101, $b$={round(b,0)}, $h$={round(h,0)}, $r_\\text{{geom}}$=3 cm, H-25, B500-S"))
display(Markdown(f"Demanda $M_u$={round(M_u,0)}, armadura {rebar_b}, $A_s$={round(A_s.magnitude,2)} cm²>$A_{{s.min}} \\color{{green}}{{\\, ✓}}$"))
display(Markdown(f"Capacidad $\phi M_n$={round(ØM_n,0)}, $\\color{{green}}{{\\text{{FU}}={round(FU_b,2)}}} \\, ✓$"))
V_u=40*kN # type: ignore # noqa: F821
rebar_v="1eØ6/20"
A_v=2.83*cm**2/m # type: ignore # noqa: F821
ØV_n=80*kN # type: ignore # noqa: F821
FU_v=V_u/ØV_n
display(Markdown(f"Demanda $V_u$={round(V_u,0)}, armadura {rebar_v}, $A_v$={round(A_v.magnitude,2)} cm²/m>$A_{{v.min}} \\color{{green}}{{\\, ✓}}$, capacidad $\phi V_n$={round(ØV_n,0)}, $\\color{{green}}{{\\text{{FU}}={round(FU_v,2)}}} \\, ✓$"))
display(Markdown(f"Capacidad $\phi V_n$={round(ØV_n,0)}, $\\color{{green}}{{\\text{{FU}}={round(FU_v,2)}}} \\, ✓$"))

Viga 101, $b$=20.00 cm, $h$=50.00 cm, $r_\text{geom}$=3 cm, H-25, B500-S

Demanda $M_u$=50.00 kN·m, armadura 3Ø12, $A_s$=3.39 cm²>$A_{s.min} \color{green}{\, ✓}$

Capacidad $\phi M_n$=55.00 kN·m, $\color{green}{\text{FU}=0.91} \, ✓$

Demanda $V_u$=40.00 kN, armadura 1eØ6/20, $A_v$=2.83 cm²/m>$A_{v.min} \color{green}{\, ✓}$, capacidad $\phi V_n$=80.00 kN, $\color{green}{\text{FU}=0.50} \, ✓$

Capacidad $\phi V_n$=80.00 kN, $\color{green}{\text{FU}=0.50} \, ✓$

In [3]:
import pandas as pd
class Formatter:
    def __init__(self):
        # Define colors in the constructor
        self.green = '#439b00'
        self.red = '#d43e36'
        self.yellow = '#efc200'
        self.mid_value=0.95
        self.max_value=1
    
    def FU(self, FU):
        # Determine color based on FU value
        if self.mid_value > FU:
            color = self.green
        elif self.mid_value <= FU <= self.max_value:
            color = self.yellow
        else:
            color = self.red
        
        return f"$\\color{{{color}}}{{\\text{{FU}}={round(FU,2)}}}$"
    def FU_value(self, FU):
        # Determine color based on FU value
        if self.mid_value > FU:
            color = self.green
        elif self.mid_value <= FU <= 1:
            color = self.yellow
        else:
            color = self.red
        
        return f"$\\color{{{color}}}{{{round(FU,2)}}}$"
    
    def is_lower(self, value1, value2):
        # Compare two values and return the appropriate formatted output
        if value1 < value2:
            return r"$\color{" + self.green + r"}{\, \checkmark}$"  # Green checkmark
        else:
            return r"$\color{" + self.red + r"}{\, \times}$"  # Red cross
    
    def is_greater(self, value1, value2):
        # Compare two values and return the appropriate formatted output
        if value1 > value2:
            return r"$\color{" + self.green + r"}{\, \checkmark}$"  # Green checkmark
        else:
            return r"$\color{" + self.red + r"}{\, \times}$"  # Red cross
    # Formatting functions outside the class
    def FU_value_df(self,FU):
        if isinstance(FU, (int, float)):
            if self.mid_value > FU:
                return f"color: {self.green}"
            elif self.mid_value <= FU <= self.max_value:
                return f"color: {self.yellow}"
            else:
                return f"color: {self.red}"
        else: 
            return ""

    def apply_FU_style(self,value):
        formatted_value = round(value, 2) if isinstance(value, (int, float)) else value
        return self.FU_value_df(formatted_value)

    def color_FU_df(self,df, fu_columns):
        """
        Apply color styling to specified FU-related columns in the DataFrame.
        
        :param df: DataFrame with FU values to color.
        :param fu_columns: List of column names to apply the FU styling to.
        :return: A styled DataFrame with colored FU values.
        """
        return df.style.map(self.apply_FU_style, subset=fu_columns).format(precision=2)

class BeamDesign:
    def __init__(self, b, h, M_u, V_u=0*kN): #type: ignore # noqa: F821
        self.b = b
        self.h = h
        self.M_u = M_u
        self.V_u = V_u
    
    A_s = 1.13*cm**2*3
    rebar_b = "3Ø12"  # Replace with actual rebar type if needed
    ØM_n = 55*kN*m  # type: ignore # noqa: F821
    A_v=2*cm**2/m #type: ignore # noqa: F821
    rebar_v="1eØ6/20"
    ØV_n = 40*kN #type:ignore # noqa: F821
    @property
    def flexure_results(self):
        FU_b = self.M_u/ØM_n  # Example Utilization Ratio (FU)
        # Create FUFormatter instance and format FU value
        formatter = Formatter()
        formatted_FU = formatter.FU(FU_b)
        # Print results
        display(Markdown(f"Armadura longitudinal inf {rebar_b}, $A_s$={round(A_s.magnitude,2)}"
                         f"cm², $M_u$={round(self.M_u,0)}, $\\phi M_n$={round(ØM_n,0)} → {formatted_FU}"))
    @property
    def shear_results(self):
        FU_v = self.V_u/ØV_n  # Example Utilization Ratio (FU)
        # Create FUFormatter instance and format FU value
        formatter = Formatter()
        formatted_FU = formatter.FU(FU_v)
        # Print results
        display(Markdown(f"Armadura transversal {rebar_v}, $A_v$={round(A_v.magnitude,2)}"
                         f"cm²/m, $V_u$={round(self.V_u,0)}, $\\phi V_n$={round(ØV_n,0)} → {formatted_FU}"))

# Example usage
b=20*cm
h=50*cm
M_u=40*kN*m # type: ignore # noqa: F821
V_u=30*kN #type:ignore # noqa: F821
beam = BeamDesign(b, h, M_u)
beam.flexure_results
M_u=54*kN*m # type: ignore # noqa: F821
beam = BeamDesign(b, h, M_u)
beam.flexure_results
M_u=70*kN*m # type: ignore # noqa: F821
beam = BeamDesign(b, h, M_u, V_u)
beam.flexure_results
beam.shear_results
formatter = Formatter()
display(Markdown(formatter.FU(0.85)))
display(Markdown(formatter.FU_value(0.85)))
display(Markdown(formatter.is_lower(0.85,1)))
display(Markdown(formatter.is_greater(0.85,1)))

# SectionSummary class
class SectionSummary:
    def __init__(self, sections):
        self.sections = sections  # List of Section objects
        self.fu_columns = ["FUb.inf", "FUb.sup", "FUv"]  # Columns to be styled based on FU value

    def create_summary_table(self):
        data = {
            "Viga": ["VF-01", "VF-02"],
            "FUb.inf": [0.9, ""],
            "FUb.sup": [0.98, 0.95],
            "FUv": [1.05, 0.92]
        }
        df = pd.DataFrame(data)
        formatter = Formatter()
        # Apply color formatting to the specified FU columns
        return formatter.color_FU_df(df, self.fu_columns)


# Example usage outside the class
summary = SectionSummary([])

# Create the summary table and apply coloring
summary.create_summary_table()


Armadura longitudinal inf 3Ø12, $A_s$=3.39cm², $M_u$=40.00 kN·m, $\phi M_n$=55.00 kN·m → $\color{#439b00}{\text{FU}=0.73}$

Armadura longitudinal inf 3Ø12, $A_s$=3.39cm², $M_u$=54.00 kN·m, $\phi M_n$=55.00 kN·m → $\color{#efc200}{\text{FU}=0.98}$

Armadura longitudinal inf 3Ø12, $A_s$=3.39cm², $M_u$=70.00 kN·m, $\phi M_n$=55.00 kN·m → $\color{#d43e36}{\text{FU}=1.27}$

Armadura transversal 1eØ6/20, $A_v$=2.83cm²/m, $V_u$=30.00 kN, $\phi V_n$=80.00 kN → $\color{#439b00}{\text{FU}=0.38}$

$\color{#439b00}{\text{FU}=0.85}$

$\color{#439b00}{0.85}$

$\color{#439b00}{\, \checkmark}$

$\color{#d43e36}{\, \times}$

,Viga,FUb.inf,FUb.sup,FUv
0,VF-01,0.90,0.98,1.05
1,VF-02,,0.95,0.92


## Ejemplo análisis de una sección

In [4]:
from mento import Concrete_ACI_318_19, SteelBar, Forces, RectangularConcreteBeam
from mento import mm, inch, kip, psi, ksi

concrete= Concrete_ACI_318_19(name="C4",f_c=4000*psi) 
steelBar= SteelBar(name="ADN 420", f_y=60*ksi) 
section = RectangularConcreteBeam(label="V-10x16",concrete=concrete,steel_bar=steelBar,width=10*inch, height=16*inch)
f = Forces(V_z=37.727*kip, N_x=0*kip)
section.set_transverse_rebar(n_stirrups=1, d_b=12*mm, s_l=6*inch) 
# results = section.check_shear(f)
section.design_shear(f)
# section.shear_design_results
section.shear_results
# section.set_transverse_rebar(n_stirrups=1, d_b=12*mm, s_l=6*inch) 

In [5]:
concrete= Concrete_ACI_318_19(name="C4",f_c=4000*psi) 
steelBar= SteelBar(name="ADN 420", f_y=60*ksi) 
custom_settings = {'clear_cover': 1.5*inch, 'stirrup_diameter_ini':0.5*inch,
                       'longitudinal_diameter_ini': 1*inch}
section = RectangularConcreteBeam(label="V-10x16",
                                    concrete=concrete,steel_bar=steelBar,width=10*inch, height=16*inch,
                                    settings=custom_settings)
f = Forces(V_z=37.727*kip, N_x=0*kip)
section.design_shear(f)
# section.shear_results

,Label,"Av,min","Av,req",Av,Vu,ØVc,ØVs,ØVn,ØVmax,Vu<ØVmax,Vu<ØVn,FUv
0,V-10x16,2.12 cm²/m,10.42 cm²/m,10.47 cm²/m,167.82 kN,56.97 kN,112.29 kN,169.26 kN,284.85 kN,True,True,0.99


In [6]:
from mento.concrete.beam import Rebar
from mento import MPa
concrete= Concrete_ACI_318_19(name="H30",f_c=30*MPa) 
steelBar= SteelBar(name="ADN 420", f_y=420*MPa) 
section = RectangularConcreteBeam(
    label="V 20x50",
    concrete=concrete,
    steel_bar=steelBar,
    width=20*cm,  
    height=50*cm,  
)
section.settings.load_aci_318_19_settings()
section.c_c = 30*mm
as_req = 0 * cm**2
beam_rebar = Rebar(section)
long_rebar_df = beam_rebar.longitudinal_rebar_ACI_318_19(A_s_req=as_req)
long_rebar_df

,n_1,d_b1,n_2,d_b2,n_3,d_b3,n_4,d_b4,total_as,total_bars,clear_spacing
0,2,12.00 mm,0,None,0,None,0,None,2.26 cm²,2,100.00 mm


In [7]:
class Section:
    def __init__(self, name, b, h, as_inf, as_sup, av, as_inf_real, as_sup_real, av_real, fiMninf, fiMnsup, fiVn, Vu, Mu, as_inf_nec, as_sup_nec, av_nec):
        self.name = name  # Beam name, e.g., 'VF-01'
        self.b = b  # Width of the beam in cm
        self.h = h  # Height of the beam in cm
        self.as_inf = as_inf  # Lower reinforcement (string)
        self.as_sup = as_sup  # Upper reinforcement (string)
        self.av = av  # Shear reinforcement (string)
        self.as_inf_real = as_inf_real  # Lower reinforcement area in cm²
        self.as_sup_real = as_sup_real  # Upper reinforcement area in cm²
        self.av_real = av_real  # Shear reinforcement area in cm²/m
        self.fiMninf = fiMninf  # Bending moment capacity in kNm
        self.fiMnsup = fiMnsup  # Bending moment capacity in kNm
        self.fiVn = fiVn  # Shear force capacity in kN
        self.Vu = Vu # Ultimate shear
        self.Mu = Mu # Ultimate bending moment
        self.as_inf_nec = as_inf_nec  # Lower reinforcement area in cm²
        self.as_sup_nec = as_sup_nec  # Upper reinforcement area in cm²
        self.av_nec = av_nec  # Shear reinforcement area in cm²/m


    def to_capacity_dict(self):
        """
        Converts beam properties to a dictionary for dataframe row.
        """
        return {
            "Viga": self.name,
            "b": self.b,
            "h": self.h,
            "As.inf": self.as_inf,
            "As.sup": self.as_sup,
            "Av": self.av,
            "As.inf.real": self.as_inf_real,
            "As.sup.real": self.as_sup_real,
            "Av.real": self.av_real,
            "MRd": self.fiMninf,
            "VRd": self.fiVn
        }
    def to_check_dict(self):
        """
        Converts beam properties to a dictionary for dataframe row.
        """
        return {
            "Viga": self.name,
            "b": self.b,
            "h": self.h,
            'Vu': self.Vu,
            'Mu': self.Mu,
            "As.inf.real": self.as_inf_real,
            "As.sup.real": self.as_sup_real,
            "Av.real": self.av_real,
            "As.inf.nec": self.as_inf_nec,
            "As.sup.nec": self.as_sup_nec,
            "Av.nec": self.av_nec,
            "MRd.inf": self.fiMninf,
            "MRd.sup": self.fiMnsup,
            "VRd": self.fiVn,
            "FUb.inf" : self.Mu/self.fiMninf, 
            "FUb.sup" : self.Mu / self.fiMnsup,
            "FUv" : self.Vu / self.fiVn 

        }

class SectionSummary:
    def __init__(self, sections):
        """
        Initializes the SectionSummary with a list of Section objects.
        
        :param sections: List of Section objects
        """
        self.capacity_columns = ["Viga", "b", "h", "As.inf", "As.sup", "Av", 
                        "As.inf.real", "As.sup.real", "Av.real", "ØMn", "ØVn"]
        self.check_columns = ["Viga", "b", "h", "Vu", "Mu", "As.inf.real", "As.inf.nec", 
                         "As.sup.real", "As.sup.nec", "Av.real", "Av.nec", "MRd.inf","MRd.sup", 
                         "VRd", "FUb.inf", "FUb.sup", "FUv"]
        self.sections = sections  # Store the list of sections

    def capacity(self):
        """
        Creates a DataFrame from the list of Section objects and adds the units row.
        
        :return: A pandas DataFrame with the data from Section objects and units as the first row.
        """
        # Convert each section object to a row dictionary
        data = [section.to_capacity_dict() for section in self.sections]
        
        # Add units row
        units_row = {
            "Viga": "",
            "b": "cm",
            "h": "cm",
            "As.inf": "",
            "As.sup": "",
            "Av": "",
            "As.inf.real": "cm²",
            "As.sup.real": "cm²",
            "Av.real": "cm²/m",
            "MRd": "kNm",
            "VRd": "kN",
        }
        
        # Combine units row with section data
        data.insert(0, units_row)
        
        # Create a DataFrame from the data
        df = pd.DataFrame(data, columns=self.capacity_columns)
        
        return df
    def check(self):
        """
        Creates a DataFrame for checking shear and bending for each section.
        Outputs necessary and real reinforcement, utilization factors for bending and shear.
        
        :return: A pandas DataFrame summarizing the check of each section.
        """

        # Convert each section object to a row dictionary
        data = [section.to_check_dict() for section in self.sections]

        # Create a DataFrame from the data
        df = pd.DataFrame(data, columns=self.check_columns)

        # Add units row
        units_row = {
            "Viga": "",
            "b": "cm",
            "h": "cm",
            "Vu": "kN",
            "Mu": "kNm",
            "As.inf.real": "cm²",
            "As.inf.nec": "cm²",
            "As.sup.real": "cm²",
            "As.sup.nec": "cm²",
            "Av.real": "cm²/m",
            "Av.nec": "cm²/m",
            "MRd.inf": "kNm",
            "MRd.sup": "kNm",
            "VRd": "kN",
            "FUb.inf": "", 
            "FUb.sup": "",
            "FUv": "" 
        }
        
        # Append the units row to the top of the DataFrame
        df = pd.concat([pd.DataFrame([units_row]), df], ignore_index=True)
        
        # Add style to FU columns
        self.fu_columns = ["FUb.inf", "FUb.sup", "FUv"]  # Columns to be styled based on FU value
        formatter = Formatter()
        # Apply color formatting to the specified FU columns
        return formatter.color_FU_df(df, self.fu_columns)
    

# Example usage:
beam1 = Section("VF-01", 20, 60, "3Ø12", "2Ø10", "1eØ6/10", 3.39, 2.01, 2, 100, 80, 10, 12,30, 2,1,1.5)
beam2 = Section("VF-02", 20, 60, "3Ø12", "2Ø10", "1eØ6/10", 3.39, 2.01, 2, 100, 80, 10, 13,30, 2,1,1.5)
beam3 = Section("V-102", 20, 60, "3Ø12", "2Ø10", "1eØ6/10", 3.39, 2.01, 2, 100, 80, 10, 9.6,30, 2,1,1.5)
beam4 = Section("V-2051", 20, 60, "3Ø12", "2Ø10", "1eØ6/10", 3.39, 2.01, 2, 100, 80, 10, 8,30, 2,1,1.5)

# Create the DataFrame
sections = [beam1, beam2, beam3, beam4]
summary = SectionSummary(sections)
df_capacity = summary.capacity()
df_check = summary.check()
df_check
# df_capacity
# df.to_excel("DF_check.xlsx")

,Viga,b,h,Vu,Mu,As.inf.real,As.inf.nec,As.sup.real,As.sup.nec,Av.real,Av.nec,MRd.inf,MRd.sup,VRd,FUb.inf,FUb.sup,FUv
0,,cm,cm,kN,kNm,cm²,cm²,cm²,cm²,cm²/m,cm²/m,kNm,kNm,kN,,,
1,VF-01,20,60,12.00,30,3.39,2,2.01,1,2,1.50,100,80,10,0.30,0.38,1.20
2,VF-02,20,60,13.00,30,3.39,2,2.01,1,2,1.50,100,80,10,0.30,0.38,1.30
3,V-102,20,60,9.60,30,3.39,2,2.01,1,2,1.50,100,80,10,0.30,0.38,0.96
4,V-2051,20,60,8.00,30,3.39,2,2.01,1,2,1.50,100,80,10,0.30,0.38,0.80


# Exportar DataFrame a Excel

In [8]:
# df_check.to_excel("DF_check.xlsx", index=False)

# Exportar DataFrame a Word

In [41]:
# Creating the first table
materials_data = {
    "Materiales": [
        "Resistencia característica del hormigón",
        "Resistencia característica del acero",
        "Peso del hormigón",
        "Módulo de deformación secante a los 28 días del hormigón"
    ],
    "Variable": ["fc", "fy", "γc", "E"],
    "Valor": [25, 420, 25, 28500],
    "Unidad": ["MPa", "MPa", "kN/m³", "MPa"]
}

df = pd.DataFrame(materials_data)

# Exporting to Excel
df.to_excel("materials_report.xlsx", index=False)

# For Word export, you can convert the DataFrame to HTML and use python-docx
from docx import Document
from docx.shared import Pt, Cm
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls

def set_col_widths(table, column_widths):
    for row in table.rows:
        for idx, width in enumerate(column_widths):
            row.cells[idx].width = width

# Initialise the Word document
doc = Document()
# Settings
style = doc.styles['Normal']
font = style.font
font.name = 'Lato'
font.size = Pt(9)
# Start writing data
doc.add_heading('Comprobación de sección de viga', level=1)
doc.add_heading('Datos', level=2)
# Initialise the table
table = doc.add_table(rows=df.shape[0]+1, cols=df.shape[1])
# Add table style
table.style = "Light Shading"
# table.allow_autofit = True
# Set the width of the columns
column_widths = [Cm(12), Cm(2), Cm(2), Cm(2)]  # Adjust widths as needed
set_col_widths(table, column_widths)
# Add the column headings
for j in range(df.shape[1]):
    table.cell(0, j).text = df.columns[j]
# Add the body of the data frame
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        cell = df.iat[i, j]
        table.cell(i + 1, j).text = str(cell)
# Change the style of the first column
for row in table.rows[1:]:
    cell = row.cells[0]  # Get the first cell in the row
    run = cell.paragraphs[0].runs[0]  # Get the first run in the cell
    run.font.bold = False  # Make the font bold

doc.add_heading('Comprobaciones mínimas', level=2)
doc.add_heading('Comprobaciones de diseño', level=2)
# Save the Word doc
doc.save("materials_report.docx")



In [1]:
from docx import Document
from docx.shared import Pt, Cm

class DocumentBuilder:
    def __init__(self, title, font_name='Lato', font_size=9):
        self.doc = Document()
        self.title = title
        self.font_name = font_name
        self.font_size = font_size
        self.set_document_style()

    def set_document_style(self):
        style = self.doc.styles['Normal']
        font = style.font
        font.name = self.font_name
        font.size = Pt(self.font_size)

    def add_heading(self, text, level):
        self.doc.add_heading(text, level=level)

    def set_col_widths(self, table, column_widths):
        for row in table.rows:
            for idx, width in enumerate(column_widths):
                row.cells[idx].width = width

    def add_table(self, df, column_widths):
        # Initialize the table
        table = self.doc.add_table(rows=df.shape[0] + 1, cols=df.shape[1])
        # Add table style
        table.style = "Light Shading"
        # Set the width of the columns
        self.set_col_widths(table, column_widths)
        # Add the column headings
        for j in range(df.shape[1]):
            table.cell(0, j).text = df.columns[j]
        # Add the body of the data frame
        for i in range(df.shape[0]):
            for j in range(df.shape[1]):
                cell = df.iat[i, j]
                table.cell(i + 1, j).text = str(cell)
        # Change the style of the first column
        for row in table.rows[1:]:
            cell = row.cells[0]  # Get the first cell in the row
            run = cell.paragraphs[0].runs[0]  # Get the first run in the cell
            run.font.bold = False  # Change font boldness (or set any style you need)

    def save(self, filename):
        self.doc.save(filename)

In [2]:
import pandas as pd

# Tabla de datos, descripción de variables.
data_general = {
    "Materiales": [
        "Resistencia característica del hormigón",
        "Resistencia característica del acero",
        "Peso del hormigón",
        "Módulo de deformación secante a los 28 días del hormigón"
    ],
    "Variable": ["fc", "fy", "γc", "E"],
    "Valor": [25, 420, 25, 28500],
    "Unidad": ["MPa", "MPa", "kN/m³", "MPa"]
}
df_general = pd.DataFrame(data_general)
column_widths_general = [Cm(12), Cm(2), Cm(2), Cm(2)]  # Adjust widths as needed

# Tabla de consideraciones mínimas y máximas
data_min_max = {
    'Comprobación': ['Cuantía geometríca', 'Cuantía mecánica'],
    'Unidad': ['cm²', 'cm²'],
    'Valor': [16, 20],
    'Min.': [15, 18],
    'Max.': [50,19],
    'Ok?': ['✔️', '❌'],
    'Referencia': ['6.5.3(a)','53.22']
}
df_min_max = pd.DataFrame(data_min_max)
column_widths_min_max = [Cm(8), Cm(2), Cm(2), Cm(2), Cm(2), Cm(1), Cm(2.5)]  # Adjust widths as needed

# Create a document builder instance
doc_builder = DocumentBuilder(title='Comprobación de sección de viga')

# Add first section and table
doc_builder.add_heading('Comprobación de sección de viga', level=1)
doc_builder.add_heading('Datos', level=2)
doc_builder.add_table(df_general, column_widths_general)

# Add second section and another table (can use different data)
doc_builder.add_heading('Comprobaciones mínimas', level=2)
doc_builder.add_table(df_min_max, column_widths_min_max)
doc_builder.add_heading('Comprobaciones de diseño', level=2)

# You can add another table here as needed
# df2 = pd.DataFrame(...)  # New DataFrame for the next section
# doc_builder.add_table(df2, column_widths)

# Save the Word doc
doc_builder.save("materials_report.docx")

### Creación del documento con Mento

In [3]:
from mento import DocumentBuilder

# Create a document builder instance
doc_builder = DocumentBuilder(title='Comprobación de sección de viga')

# Add first section and table
doc_builder.add_heading('Comprobación de sección de viga', level=1)
doc_builder.add_heading('Datos', level=2)
doc_builder.add_table(df_general, column_widths_general)

# Add second section and another table (can use different data)
doc_builder.add_heading('Comprobaciones mínimas', level=2)
doc_builder.add_table(df_min_max, column_widths_min_max)
doc_builder.add_heading('Comprobaciones de diseño', level=2)

# Save the Word doc
doc_builder.save("materials_report.docx")

## Resultado Detallado en Jupyter

In [62]:
from tabulate import tabulate

# Display in Jupyter Notebook
table_general = tabulate(
    data_general,
    headers='keys',
    tablefmt="fancygrid",
    numalign="right",
    colalign=("left", "center", "right","left")
)
print("======= DATOS GENERALES ======= \n",table_general,'\n')
table_min_max = tabulate(
    data_min_max,
    headers='keys',
    tablefmt="fancygrid",
    numalign="right",
    colalign=("left", "center", "center","center","center","center","left")
)
print("======= COMPROBACIONES MÍNIMAS Y MÁXIMAS ======= \n", table_min_max, '\n')

======= DATOS GENERALES ======= 
 Materiales                                                 Variable     Valor  Unidad
--------------------------------------------------------  ----------  -------  --------
Resistencia característica del hormigón                       fc           25  MPa
Resistencia característica del acero                          fy          420  MPa
Peso del hormigón                                             γc           25  kN/m³
Módulo de deformación secante a los 28 días del hormigón      E         28500  MPa 

======= COMPROBACIONES MÍNIMAS Y MÁXIMAS ======= 
 Comprobación         Unidad    Valor    Min.    Max.    Ok?   Referencia
------------------  --------  -------  ------  ------  -----  ------------
Cuantía geometríca    cm²       16       15      50     ✔️    6.5.3(a)
Cuantía mecánica      cm²       20       18      19     ❌    53.22 



In [1]:
from tabulate import tabulate

class TablePrinter:
    def __init__(self, title=None):
        self.title = title

    def print_table_data(self, data, headers, tablefmt="fancygrid", numalign="right"):
        if self.title:
            print(f"======= {self.title} ======= \n")
        
        colalign=("left", "center", "right","left")
        table = tabulate(
            data,
            headers=headers,
            tablefmt=tablefmt,
            numalign=numalign,
            colalign=colalign
        )
        print(table, '\n')
    def print_table_min_max(self, data, headers, tablefmt="fancygrid", numalign="right"):
        if self.title:
            print(f"======= {self.title} ======= \n")
        
        colalign=("left", "center", "center", "center", "center", "center", "left")
        table = tabulate(
            data,
            headers=headers,
            tablefmt=tablefmt,
            numalign=numalign,
            colalign=colalign
        )
        print(table, '\n')

# Example usage
data_general = {
    'Materiales': ['Resistencia característica del hormigón', 'Resistencia característica del acero'],
    'Variable': ['fc', 'fy'],
    'Valor': [25, 420],
    'Unidad': ['MPa', 'MPa']
}

data_min_max = {
    'Comprobaciones': ['Cuantía mínima geométrica', 'Cuantía mínima mecánica'],
    'Unidad': ['cm²', 'cm²'],
    'Valor': [16, 16],
    'Min.': [15, 15],
    'Máx.': [200, 200],
    'Ok?': ['✔️', '❌'],
    'Referencia': ['6.5.3(a)', '6.5.3(a)']
}

# Create a TablePrinter instance and display tables
general_printer = TablePrinter("DATOS GENERALES")
general_printer.print_table_data(data_general, headers='keys')

min_max_printer = TablePrinter("COMPROBACIONES MÍNIMAS Y MÁXIMAS")
min_max_printer.print_table_min_max(data_min_max, headers='keys')

======= DATOS GENERALES ======= 

Materiales                                Variable     Valor  Unidad
---------------------------------------  ----------  -------  --------
Resistencia característica del hormigón      fc           25  MPa
Resistencia característica del acero         fy          420  MPa 

======= COMPROBACIONES MÍNIMAS Y MÁXIMAS ======= 

Comprobaciones              Unidad    Valor    Min.    Máx.    Ok?   Referencia
-------------------------  --------  -------  ------  ------  -----  ------------
Cuantía mínima geométrica    cm²       16       15     200     ✔️    6.5.3(a)
Cuantía mínima mecánica      cm²       16       15     200     ❌    6.5.3(a) 



### Resumen detallado con Mento

In [2]:
from mento import TablePrinter

# Create a TablePrinter instance and display tables
general_printer = TablePrinter("DATOS GENERALES")
general_printer.print_table_data(data_general, headers='keys')

min_max_printer = TablePrinter("COMPROBACIONES MÍNIMAS Y MÁXIMAS")
min_max_printer.print_table_min_max(data_min_max, headers='keys')

======= DATOS GENERALES ======= 

Materiales                                Variable     Valor  Unidad
---------------------------------------  ----------  -------  --------
Resistencia característica del hormigón      fc           25  MPa
Resistencia característica del acero         fy          420  MPa 

======= COMPROBACIONES MÍNIMAS Y MÁXIMAS ======= 

Comprobaciones              Unidad    Valor    Min.    Máx.    Ok?   Referencia
-------------------------  --------  -------  ------  ------  -----  ------------
Cuantía mínima geométrica    cm²       16       15     200     ✔️    6.5.3(a)
Cuantía mínima mecánica      cm²       16       15     200     ❌    6.5.3(a) 

